## LangChain: Models, Prompts and Output Parsers


## Outline :

 * Direct API calls to OpenAI/Gemini Pro/Lamma3
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

## First Get your [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [ ]:
## Load you env variables
import os
from dotenv import load_dotenv,find_dotenv
import openai
import anthropic
from anthropic import Anthropic

_ = load_dotenv(find_dotenv(filename='.env'))
print("your openai api key is {OPENAI_API_KEY}".format(OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")))
print("your claud api key is {ANTHROPIC_API_KEY}".format(ANTHROPIC_API_KEY=os.getenv("ANTHROPIC_API_KEY")))

## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

In [3]:
def openai_call(prompt,model="gpt-4o"):
    messages=[
        {
            "role":"user",
            "content":prompt
        }
    ]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.2
    )
    return response.choices[0].message.content

In [4]:
openai_call("What is Langchain ?")

'LangChain is a framework designed to facilitate the development of applications that leverage large language models (LLMs). It provides a suite of tools and abstractions that make it easier to build complex applications that can interact with LLMs in a meaningful way. LangChain is particularly useful for applications that require:\n\n1. **Data Augmented Generation**: Enhancing the capabilities of LLMs by integrating external data sources.\n2. **Agentic Behavior**: Allowing LLMs to perform actions, interact with their environment, and make decisions based on the context.\n3. **Memory**: Enabling LLMs to remember past interactions and use this memory to inform future responses.\n\nLangChain supports various components to achieve these functionalities:\n\n- **Prompt Templates**: Standardized templates for generating prompts that can be used to query LLMs.\n- **Chains**: Sequences of calls to LLMs or other utilities that can be combined to perform complex tasks.\n- **Indexes**: Structures

In [5]:
## Design custom prompt basesd on specific user message
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey
"""
style = """Dialect Darija with respectfull language"""
designed_prompt = f"""
Translate in arabic darija the text that is delimited by triple backricks \
with a style that is {style}.
text:```{customer_email}``` \
"""

In [6]:
## get the formated prompt
print(designed_prompt)


Translate in arabic darija the text that is delimited by triple backricks with a style that is Dialect Darija with respectfull language.
text:```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey
``` 


In [7]:
openai_call(prompt=designed_prompt)

'```darija\nآآآه، راه أنا معصب بزاف حيث غطا ديال البلندر طار ووسخ لي حيطان المطبخ ديالي بالسmoothie! وزيد عليها، الضمانة ما كاتغطيش تكاليف تنظيف المطبخ ديالي. خصني مساعدتك دابا، صاحبي.\n```'

In [8]:
claud_client = Anthropic(
    api_key=os.getenv("ANTHROPIC_API_KEY")
)

In [9]:
def claudai_call(prompt,model="claude-3-opus-20240229"):
    input_messages=[
        {
            "role":"user",
            "content":"Hello claud , can you assist me on this conversation !",
            "role":"assistant",
            "content":"of course , let me know about your question",
            "role":"user",
            "content":prompt
        }
    ]
    claud_response=claud_client.messages.create(
        model=model,
        max_tokens=1024,
        messages=input_messages,
        temperature=0.1
    )
    return claud_response.content


In [ ]:
claudai_call(prompt=designed_prompt)

## Chat API : LangChain Framework

Let's try how we can do the same using LangChain.

In [11]:
## Instantiate the OpenAI Langchain Wrapper
from langchain_openai import ChatOpenAI

### Used Model

In [12]:
openai_chat_model = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.2,
    streaming=True,
    api_key=os.getenv("OPENAI_API_KEY"))
openai_chat_model

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001FD286F6680>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001FD286F7D90>, temperature=0.2, openai_api_key=SecretStr('**********'), openai_proxy='', streaming=True)

In [16]:
## Define the custom Template 
style = "structured and readable language"
text = "Hello everyone your are in the langchain tutorial . Happy Learning"
template_string= """
Translate me the text that is delimited by triple backticks into arabic .\
With the following style : {style}
text: ```{text}```
"""

print(template_string)


Translate me the text that is delimited by triple backticks into arabic .With the following style : {style}
text: ```{text}```



### Langchain Prompt Template

In [17]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string)
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [19]:
customer_messages = prompt_template.format_messages(
   style=style,
   text=text
)
customer_messages

[HumanMessage(content='\nTranslate me the text that is delimited by triple backticks into arabic .With the following style : structured and readable language\ntext: ```Hello everyone your are in the langchain tutorial . Happy Learning```\n')]

In [28]:
## Call the LLM for generating the response based on the prompt template
customer_response = openai_chat_model.invoke(customer_messages)
customer_response.content

'مرحبًا بالجميع، أنتم الآن في دورة تعليم لغة البرمجة. نتمنى لكم تعلمًا ممتعًا.'

## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [29]:
## structure of the final output
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [30]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [36]:
customer_review_template = ChatPromptTemplate.from_template(
    review_template
)
customer_review_messages = customer_review_template.format_messages(text=customer_review)
llm__review = openai_chat_model.invoke(customer_review_messages)
llm__review.content

'{\n    "gift": true,\n    "delivery_days": 2,\n    "price_value": ["It\'s slightly more expensive than the other leaf blowers out there"]\n}'

In [37]:
# check the type of the generated response
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
llm__review.content.get('gift')

str

In [45]:
## Output Parser for more suitable type of responses
from langchain_core.output_parsers.json import JsonOutputParser
jsonParser = JsonOutputParser()
jsonparsed_llm_review = jsonParser.invoke(llm__review.content)
type(jsonparsed_llm_review)

dict

In [44]:
jsonparsed_llm_review.get('price_value')

["It's slightly more expensive than the other leaf blowers out there"]